# Análise baseada em fatores dos 10 FIAs livres brasileiros com maiores patrimônios líquidos
**Escrito por**: Enrique Quintslr (https://github.com/enriquemq/)  
 <font size="2"> *Qualquer crítica, dúvida ou sugestão, sinta-se a vontade para entrar em contato pelo meio de sua preferência.*</font> 


## Introdução

É natural esperar que os fundos de investimento em ações melhores sucedidos em arrecadação de recursos sejam também aqueles com os gestores mais competentes. Este notebook busca estudar essa hipótese através de um exercício baseado em Fama e French (2010). Resumidamente,  usando dados diários dos 10 maiores FIAs livres em patrimônio líquido com mais de 2 anos de atividade, é feita a regressão do retornos dos fundos com os retornos dos principais fatores de risco para verificar a existência de retornos anormais (mais conhecido como $\alpha$).
O notebook está organizado da seguinte forma: primeiro apresenta a metodologia, em seguida apresenta o código utilizado e, por fim, os resultados são expostos e analisados.

## Dados e metodologia

**Seleção dos fundos**:  
Usando a plataforma do https://maisretorno.com/, foi obtida a lista dos 10 fundos com maior patrimônio líquido entre os fundos de ações livres e com pelo menos dois anos de atividade.

**Dados**:  
Os dados dos fundos foram extraídos do site https://infofundos.com.br/ e os dados dos fatores de risco foram obtidos através das séries calculadas pelo NEFIN (http://nefin.com.br/).

**Metodologia:**  
Baseado em Fama e French (2010), com o objetivo de encontrar indícios de retornos anormais, para cada fundo $i$ é feita a seguinte regressão:  

$$r_{i, t} - r_{f, t} = \alpha_{i} + \beta_{i,1}(r_{mkt, t} - r_{f, t}) + \beta_{i,2}SMB_t + \beta_{i,3}HML_t + \beta_{i,4}WML_t + \beta_{i, 5}IML_t + \epsilon_{i,t}$$
  
Onde $r_{i}$ é o retorno diário do fundo $i$, $r_f$ é a taxa livre de risco, $r_{mkt}$ é retorno diário do Ibovespa, $SMB$ é o retorno diário do portfólio teórico de tamanho, $HML$ é o retorno diário do portfólio teórico de valor, $WML$ é o retorno diário do portfólio teórico de momentum e $IML$ é o retorno diário do portfólio teórico de iliquidez. Note que em Fama e French (2010) são usados apenas os 4 primeiros fatores de risco, diferente do que é feito neste notebook.  
Diremos que um fundo apresenta retorno anormal se o $\hat{\alpha}$ estimado tiver significância estatística e for positivo.
Para podermos fazer comparações, a regressão será feita com o mesmo corte temporal para todos os fundos, sendo este de 05/03/2018 até 29/05/2020.


## Código

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from statsmodels.formula.api import ols

In [ ]:
# Pegar e limpar data frame com os dados dos fundos e juntar com os dados dos fatores de risco
def montar_base_regressao(base_fundos, dicionario_base_ff, base_risk_free):
  
  dfs_factors = dicionario_base_ff

  base_fundos = base_fundos[['Fundo', 'Data', 'Variação']]
  base_fundos = base_fundos.set_index('Data')

  base_fundos_unstacked = base_fundos.pivot(columns='Fundo', values = 'Variação')
  base_fundos_unstacked = base_fundos_unstacked.dropna()
  
  base_regressao = base_fundos_unstacked
  for key in dfs_factors.keys():
    base_regressao = base_regressao.merge(dfs_factors[key], how = 'inner', on = 'Data')
  base_regressao = base_regressao.merge(base_risk_free, how = 'inner', on = 'Data')
  base_regressao = base_regressao.dropna()
  print('Janela de regressão de ', base_regressao.index[0].strftime("%Y-%m-%d"), 'até ', base_regressao.index[-1].strftime("%Y-%m-%d"), '.')
  return base_regressao

In [ ]:
# Rodar a regressão fundo ~ fatores e recuperar os parâmetros e os p-valores
def resultados_ols(coluna, base_regressao):
  base_regressao['Y'] = base_regressao.iloc[:,coluna] - base_regressao['risk_free']
  model = ols("Y ~ Rm_minus_Rf + SMB + HML + WML + IML", base_regressao).fit()
  p_val = np.round(model.pvalues,5)
  p_val.index = p_val.index + '_pval'
  p_val
  
  params = model.params

  linha = params.append(p_val)

  df = pd.DataFrame(linha)
  df.columns = [str(base_regressao.columns[coluna])]
  df = df.transpose()
  return  df

In [ ]:
# Montar um dataframe no qual cada linha é um FIA e as colunas são os parametros e os p valores das regressoes
def montar_base_resultados(base_regressao):
  df = pd.DataFrame()
  for i in range(base_regressao.shape[1] - 6):
    df_aux = resultados_ols(i, base_regressao)
    df = pd.concat([df, df_aux])
  return df


In [ ]:
# Juntando tudo
def analise_risk_factors(base_fundos, dicionario_base_ff, risk_free):
  base_regressao = montar_base_regressao(df, dfs_factors, risk_free)
  base_resultados = montar_base_resultados(base_regressao)
  return base_resultados

## Resultado e análise

### Resultado

In [ ]:
# Baixando os dados e rodando a função acima
dados = files.upload()
df = pd.read_excel(dados['top_PL_FIAs_2anos.xlsx'])
base_risk_free = pd.read_excel(dados['base_risk_free.xlsx'])
dfs_factors = obter_risk_factors_nefin() # função que baixa os dados do site do nefin e consolida em um dicionário
resultados = analise_risk_factors(df, dfs_factors, base_risk_free)
resultados

Janela de regressão de  2018-03-05 até  2020-05-29 .


,Intercept,Rm_minus_Rf,SMB,HML,WML,IML,Intercept_pval,Rm_minus_Rf_pval,SMB_pval,HML_pval,WML_pval,IML_pval
ATMOS MASTER FUNDO DE INVESTIMENTO DE AÇÕES,0.000924,0.923092,0.419935,-0.316484,0.155973,-0.021070,0.00121,0.0,0.00000,0.00000,0.00000,0.77833
BRASIL CAPITAL 30 MASTER FUNDO DE INVESTIMENTO EM AÇÕES,0.000568,0.908945,0.397385,-0.333955,0.191332,0.035747,0.02575,0.0,0.00000,0.00000,0.00000,0.59359
DYNAMO COUGAR FUNDO DE INVESTIMENTO EM AÇÕES,0.000696,0.859589,0.359493,-0.266901,0.148145,-0.094350,0.02039,0.0,0.00000,0.00000,0.00003,0.23156
ITAÚ DUNAMIS MASTER FUNDO DE INVESTIMENTO EM AÇÕES,0.000373,0.878547,0.212666,-0.049487,0.194089,-0.063875,0.09247,0.0,0.00007,0.10694,0.00000,0.27392
MOAT CAPITAL FUNDO DE INVESTIMENTO EM AÇÕES MASTER,0.001011,1.063548,0.445493,-0.017534,-0.034866,-0.274289,0.00117,0.0,0.00000,0.68273,0.34112,0.00083
NÚCLEO MASTER FUNDO DE INVESTIMENTO DE AÇÕES,0.000522,0.946566,0.346391,-0.448803,0.257755,-0.008327,0.08122,0.0,0.00000,0.00000,0.00000,0.91573
OPPORTUNITY LÓGICA MASTER FUNDO DE INVESTIMENTO EM AÇÕES,0.000501,0.921737,0.138049,-0.127712,0.169037,0.049728,0.02703,0.0,0.01140,0.00005,0.00000,0.40376
RT CONSTELLATION FUNDO DE INVESTIMENTO EM AÇÕES,0.000235,0.927742,0.145973,-0.057708,0.072285,-0.075496,0.08521,0.0,0.00001,0.00231,0.00001,0.03580
TRUXT VALOR MASTER FUNDO DE INVESTIMENTO EM AÇÕES,0.000931,0.869554,0.170367,-0.206120,0.180412,-0.045071,0.00150,0.0,0.01556,0.00000,0.00000,0.55773
VELT MASTER FUNDO DE INVESTIMENTO EM AÇÕES,0.000809,0.811753,0.318161,-0.399598,0.190944,-0.015934,0.00121,0.0,0.00000,0.00000,0.00000,0.80769


### Análise

Com valor crítico de $0.01$, temos que 4 entre os 10 fundos apresentam $\hat{\alpha}$ positivo e estatisticamente significante, sendo esses o Atmos Master, Moat Capital, Truxt Valor Master e Velt Master. Já usando um valor crítico de $0.05$, Brasil Capital 30, Dynamo Cougar e Opportunity Lógica também passam a apresentar $\hat{\alpha}$ positivo e estatisticamente significante. Por fim, usando um valor crítico de $0.1$ observamos que todos os dez fundos apresentam $\hat{\alpha}$ estatisticamente significativo e positivo. Desta forma, o resultado mostra que os fundos melhores sucedidos comercialmente apresentam indícios de habilidade dos gestores, indicando um certa eficiência do mercado em alocar recursos nas mãos de gestores (pelo menos aparentemente) mais habilidosos.  

Agora analisando os outros coeficientes, temos que os dez fundos analisados apresentam market beta próximo de $1$ e estatisticamente significante com valor crítico de $0.01$. Tal resultado era esperado uma vez que todos são fundos de ações.  Analisando a exposição dos fundos ao portfólio de tamanho, observa-se que todos tem seus parâmetros referentes ao $SMB$ positivos e estatisticamente significativos, mostrando que seus gestores capturaram o prêmio de risco relacionado ao investimento em empresas relativamente menores. No outro extremo, apenas dois fundos apresentam coeficiente referente ao portfólio de iliquidez significativo estatisticamente e em ambos os casos esse valor é negativo.  

Um resultado interessante é que os 10 fundos têm coeficiente relacionado ao fator de risco de valor ($HML$) negativo, sendo 8 estatisticamente significantes. Como o portfólio de valor tem performado mal no passado recente faz sentido que gestores ativos bem sucedidos estejam 'short' em valor.  

Ainda, 9 entre os 10 fundos tem coeficiente relacionado ao momentum ($WML$) positivo e estatisticamente significativo mesmo com valor crítico de $0.01$. Esse resultado é coerente, uma vez que o fator de risco de momentum é sabidamente o fator que apresenta melhor performance histórica. 

Por fim, é importante ressaltar que **$\hat{\alpha}$ positivo é apenas um indício de habilidade**, é possível que um fundo apresente  $\hat{\alpha}$ positivo e significante por acaso. O artigo de referência apresenta uma ótima discussão sobre o tema "habilidade versus sorte", sendo sua leitura recomendada para os leitores mais interessados.

## Referência:
- **Fama, Eugene F., and Kenneth R. French**. 2010.
“Luck versus Skill in the Cross-Section of Mutual
Fund Returns.” Journal of Finance 65(5): 1915 – 47 (https://onlinelibrary.wiley.com/doi/abs/10.1111/j.1540-6261.2010.01598.x)